## Data Ingestion

In [1]:
import pandas as pd

column_names = ['id', 'text', 'label']
weibo_train = pd.read_csv("/kaggle/input/weibo-dataset/weibo.train", sep="\t", 
                               header=None, names=column_names)
weibo_test = pd.read_csv("/kaggle/input/weibo-dataset/weibo.test", sep="\t", 
                              header=None, names=column_names)
weibo_dev = pd.read_csv("/kaggle/input/weibo-dataset/weibo.dev", sep="\t", 
                             header=None, names=column_names)

In [2]:
weibo_test.dropna(inplace=True)

In [3]:
weibo_train.head()

,id,text,label
0,3910552407639880,【教育部新规拟要求高校辅导员必须是中共党员】教育部日前在官网公布了《普通高等学校辅导员队伍建...,non-rumor
1,3536960792696376,【民生】特大好消息：继北京出台95岁老人看病不要钱后，山东泰安宣布100岁可免费登泰山，初步...,false
2,3921542276460163,【被疯传的投影舞 看哭了[泪]】这段不足4分钟的舞蹈由8位舞者共同完成，舞出一对情侣从相识到...,non-rumor
3,3922114622540868,#突发#【深圳一处工业园区被曝遭山体滑坡掩埋】据网友曝料，深圳光明新区红坳村柳溪工业园发生山...,non-rumor
4,3911293176451011,【“朝阳群众”注册人数已达13万！[赞]】他们大隐隐于市，不露声色却屡建奇功……目前登记在册...,non-rumor


In [4]:
weibo_train.shape, weibo_test.shape, weibo_dev.shape

((3147, 3), (1049, 3), (467, 3))

In [27]:
weibo_train['label'].value_counts()

label
non-rumor    1587
false        1560
Name: count, dtype: int64

In [28]:
weibo_test['label'].value_counts()

label
non-rumor    529
false        520
Name: count, dtype: int64

In [29]:
weibo_dev['label'].value_counts()

label
non-rumor    235
false        232
Name: count, dtype: int64

## Data Preparation

In [5]:
LABELS = weibo_train['label'].unique().tolist()
train_label = weibo_train['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/2429078884.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = weibo_train['label'].replace(LABELS, [0, 1]).tolist()


In [6]:
LABELS

['non-rumor', 'false']

In [7]:
train_data = weibo_train['text'].tolist()

In [8]:
dev_data = weibo_dev['text'].tolist()
dev_label = weibo_dev['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/358168363.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = weibo_dev['label'].replace(LABELS, [0, 1]).tolist()


In [9]:
test_data = weibo_test['text'].tolist()
test_label = weibo_test['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/6925416.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = weibo_test['label'].replace(LABELS, [0, 1]).tolist()


In [10]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [11]:
from transformers import BertTokenizer, BertModel, AdamW

max_length = 128

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [13]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [14]:
num_classes = 2

In [15]:
class MBERTClassifier(nn.Module):
    def __init__(self, mbert_model, num_classes, dropout=0.1):
        super(MBERTClassifier, self).__init__()
        self.mbert = mbert_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.mbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [16]:
mbert_model = BertModel.from_pretrained('bert-base-multilingual-cased')

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [17]:
model = MBERTClassifier(mbert_model, num_classes)

In [18]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

MBERTClassifier(
  (mbert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    if num_classes == 2:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred)
        }
    else:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='weighted'),
            'recall': recall_score(y_true, y_pred, average='weighted'),
            'f1': f1_score(y_true, y_pred, average='weighted')
        }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [20]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [21]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [22]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 99/99 [00:40<00:00,  2.46it/s]



Epoch 1/5
Training Metrics:
Loss: 0.5672
Accuracy: 0.7077
Precision: 0.7116
Recall: 0.6897
F1: 0.7005
Roc_auc: 0.7858

Validation Metrics:
Accuracy: 0.8544
Precision: 0.8203
Recall: 0.9052
F1: 0.8607
Roc_auc: 0.9400


Epoch 2/5: 100%|██████████| 99/99 [00:39<00:00,  2.50it/s]



Epoch 2/5
Training Metrics:
Loss: 0.3389
Accuracy: 0.8675
Precision: 0.8647
Recall: 0.8686
F1: 0.8666
Roc_auc: 0.9306

Validation Metrics:
Accuracy: 0.9101
Precision: 0.8740
Recall: 0.9569
F1: 0.9136
Roc_auc: 0.9721


Epoch 3/5: 100%|██████████| 99/99 [00:39<00:00,  2.50it/s]



Epoch 3/5
Training Metrics:
Loss: 0.1882
Accuracy: 0.9349
Precision: 0.9329
Recall: 0.9359
F1: 0.9344
Roc_auc: 0.9769

Validation Metrics:
Accuracy: 0.8972
Precision: 0.8485
Recall: 0.9655
F1: 0.9032
Roc_auc: 0.9699


Epoch 4/5: 100%|██████████| 99/99 [00:39<00:00,  2.50it/s]



Epoch 4/5
Training Metrics:
Loss: 0.1398
Accuracy: 0.9520
Precision: 0.9479
Recall: 0.9558
F1: 0.9518
Roc_auc: 0.9859

Validation Metrics:
Accuracy: 0.9272
Precision: 0.9541
Recall: 0.8966
F1: 0.9244
Roc_auc: 0.9763


Epoch 5/5: 100%|██████████| 99/99 [00:39<00:00,  2.50it/s]



Epoch 5/5
Training Metrics:
Loss: 0.0692
Accuracy: 0.9797
Precision: 0.9783
Recall: 0.9808
F1: 0.9795
Roc_auc: 0.9955

Validation Metrics:
Accuracy: 0.9251
Precision: 0.8924
Recall: 0.9655
F1: 0.9275
Roc_auc: 0.9808


In [24]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,0.567205,0.707658,0.711640,0.689744,0.700521,0.785837
1,0.338892,0.867493,0.864710,0.868590,0.866645,0.930649
2,0.188173,0.934859,0.932907,0.935897,0.934400,0.976942
3,0.139801,0.952018,0.947870,0.955769,0.951803,0.985899
4,0.069209,0.979663,0.978261,0.980769,0.979513,0.995494


In [25]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.854390,0.820312,0.905172,0.860656,0.940040
1,0.910064,0.874016,0.956897,0.913580,0.972065
2,0.897216,0.848485,0.965517,0.903226,0.969883
3,0.927195,0.954128,0.896552,0.924444,0.976339
4,0.925054,0.892430,0.965517,0.927536,0.980759


In [26]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.9122974261201144,
 'precision': 0.8794326241134752,
 'recall': 0.9538461538461539,
 'f1': 0.915129151291513,
 'roc_auc': 0.9754180602006689}